In [33]:
import requests
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go

1. The pandemic has shifted consumers' preferences away from services and towards goods, in particular durable goods.
2. One way to look at it is to observe the supply curves for goods and services.
3. The Bureau of Economic Analysis (BEA) provides such a information in its NIPA tables (T20303 and T20304) on a quarterly basis for the United States
4. In what follows, I attempt to reproduce Figure 2 from 
Konczal, M., 2023. Inflation in 2023: Causes, Progress, and Solutions. https://rooseveltinstitute.org/wp-content/uploads/2023/03/RI_MikeKonczal_HouseTestimony_202303.pdf

### The first step is to set up the API to download data from BEA

In [ ]:
parameters = {
    "UserID": 'myAPIkey',
    "Method": "GetData",
    "Datasetname": "NIPA",
    "TableName": "T20303",  # T20304
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"
}
response = requests.get('https://apps.bea.gov/api/data/', params=parameters)
data = response.json()["BEAAPI"]["Results"]["Data"]

In [38]:
parameters = {
    "UserID": '9452074C-9BA9-4108-B38D-55B924516C1C',
    "Method": "GetData",
    "Datasetname": "NIPA",
    "TableName": "T20303",
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"
}
response = requests.get('https://apps.bea.gov/api/data/', params=parameters)

data = response.json()["BEAAPI"]["Results"]["Data"]
df_quantity = pd.DataFrame(data)
df_quantity["DataValue"] = pd.to_numeric(df_quantity["DataValue"])
df_quantity = df_quantity.pivot_table(index="TimePeriod", columns="LineDescription", values="DataValue")

parameters = {
    "UserID": '9452074C-9BA9-4108-B38D-55B924516C1C',
    "Method": "GetData",
    "Datasetname": "NIPA",
    "TableName": "T20304",
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"
}
response = requests.get('https://apps.bea.gov/api/data/', params=parameters)

data = response.json()["BEAAPI"]["Results"]["Data"]
df_price = pd.DataFrame(data)
df_price["DataValue"] = pd.to_numeric(df_price["DataValue"])
df_price = df_price.pivot_table(index="TimePeriod", columns="LineDescription", values="DataValue")

# Merge price and quantities into a dataframe
df_pq = df_quantity[['Goods', 'Services']]
df_pq = df_pq.rename(columns = {'Goods': 'Quantity (goods)', 'Services': 'Quantity (services)'})
df_pq['Price (goods)'] = df_price['Goods']
df_pq['Price (services)'] = df_price['Services']

#### This shows the content of the dataset in the last n periods 

In [39]:
latest_n_values = 18
df_pq_latest = df_pq[-latest_n_values:]
df_pq_latest.head(latest_n_values)

LineDescription,Quantity (goods),Quantity (services),Price (goods),Price (services)
TimePeriod,,,,
2018Q3,126.095,112.428,95.335,115.433
2018Q4,127.114,112.732,95.095,116.198
2019Q1,127.100,112.895,94.785,116.721
2019Q2,128.801,113.265,95.216,117.519
2019Q3,130.630,113.922,94.877,118.159
2019Q4,131.544,114.519,94.913,118.765
2020Q1,131.538,111.890,94.727,119.509
2020Q2,127.857,98.311,93.416,119.511
2020Q3,142.706,106.388,94.345,120.416


### Finally, let's visualize the data in scatter plots

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Goods", "Services"),
                    column_widths=[0.5, 0.5],
                    horizontal_spacing=0.1,
                    specs=[[{"type": "scatter"}, {"type": "scatter"}]],
                    )

fig.add_trace(
    go.Scatter(x=df_pq_latest['Quantity (goods)'],
               y=df_pq_latest['Price (goods)'],
               mode='lines+markers+text',
               name='goods',
               # text=df_pq_latest.index,
               text=[index if i%2==0 else '' for i, index in enumerate(df_pq_latest.index)],
               textposition="top center",
               textfont=dict(size=10),
               marker=dict(size=8, line=dict(width=1, color="black")),
               showlegend=False),
    row=1, col=1
)

fig.update_xaxes(title_text="Quantity (2012=100)",
                 showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=1)
fig.update_yaxes(title_text="Price (2012=100)",
                 showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=1)

fig.add_trace(
    go.Scatter(x=df_pq_latest['Quantity (services)'],
               y=df_pq_latest['Price (services)'],
               mode='lines+markers+text',
               name='services',
            #    text=df_pq_latest.index,
               text=[index if i%2==0 else '' for i, index in enumerate(df_pq_latest.index)],
               textposition="top left",
               textfont=dict(size=10),
               marker=dict(size=8, line=dict(width=1, color="black")),
               showlegend=False),
    row=1, col=2
)

fig.update_xaxes(title_text="Quantity (2012=100)",
                 showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=2)
fig.update_yaxes(showgrid=True, gridcolor='lightgray', zeroline=False,
                 row=1, col=2)


fig.update_layout(height=500, width=1200,
                  margin=dict(l=50, r=50, t=50, b=50),
                  font=dict(family="Arial", size=12, color="black"),
                  plot_bgcolor='white',
                  )
fig.show()